In [19]:
import os
import re
import pandas as pd
path = 'C:/Users/kanya/Desktop/Documents/texttransfer data'
subfolder = os.listdir(path)


all_docs = []
domains = []
projects = []
files = []
for i in subfolder:
    for j in os.listdir(path+'/'+ i):
        with open (path+'/'+ i + '/' + j, "r", encoding="utf-8") as myfile:
            data=myfile.readlines()
            data = ' '.join(data)
        all_docs += [data]
        domains += [i]
        ind = [m.start() for m in re.finditer('_', j)]
        files += [j]
        projects += [j[ind[-2]+1:ind[-1]]]

In [21]:
data = pd.DataFrame({'Domain': domains,
                     'Projects': projects,
                     'file':files,
                     'text': all_docs})

In [23]:
data['text1'] = data['text'].str.replace('-\n ', '').str.replace('-\n', '')
data['text1'] = data['text1'].apply(lambda x: re.sub('\s+pagebreak', ' PAGEBREAK wozhizhi', x))
data['text1'] = data['text1'].str.replace('\n \n', ' wozhidaode').str.replace('\n  \n', ' wozhidaode').str.replace('\n \d+', ' wozhidaode').str.replace('\n\d+', ' wozhidaode').str.replace('\n', '').str.replace('wozhidaode', '.\n \n')
data['text2'] = data['text1'].str.replace('. \n ', '.\n ').str.replace('.\n ', 'wozhizhi').str.split('wozhizhi')

C:\Users\kanya\AppData\Local\Temp/ipykernel_277340/3152228529.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text1'] = data['text1'].str.replace('\n \n', ' wozhidaode').str.replace('\n  \n', ' wozhidaode').str.replace('\n \d+', ' wozhidaode').str.replace('\n\d+', ' wozhidaode').str.replace('\n', '').str.replace('wozhidaode', '.\n \n')
C:\Users\kanya\AppData\Local\Temp/ipykernel_277340/3152228529.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text2'] = data['text1'].str.replace('. \n ', '.\n ').str.replace('.\n ', 'wozhizhi').str.split('wozhizhi')


In [24]:
for i in range(len(data)):
    if i == 0:
        re_data = pd.DataFrame({'text': data['text2'][i]})
        re_data['Domain'] = data['Domain'][i]
        re_data['Projects'] = data['Projects'][i]
        re_data['file'] = data['file'][i]
    else:
        re_data1 = pd.DataFrame({'text': data['text2'][i]})
        re_data1['Domain'] = data['Domain'][i]
        re_data1['Projects'] = data['Projects'][i]
        re_data1['file'] = data['file'][i]
        re_data = pd.concat([re_data, re_data1])

In [25]:
re_data.index = range(len(re_data))
re_data['text'] = re_data['text'].str.replace('\s+', ' ', regex=True)

In [26]:
re_data['len'] = re_data['text'].apply(lambda x: len(x))
re_data = re_data[re_data['len'] >6]
re_data.index = range(len(re_data))

In [27]:
re_data['text'] = re_data['text'].apply(lambda x: x[1:] if x[0] == ' ' else x)
re_data['text'] = re_data['text'].apply(lambda x: x[:-1] if x[-1] == ' ' else x)

In [28]:
re_data['len'] = re_data['text'].apply(lambda x: len(x))
re_data = re_data[re_data['len'] >5]
re_data.index = range(len(re_data))

In [30]:
re_data['text4model'] = re_data['text']

In [31]:
tokens = re_data['text4model'].str.split(' ')
for i in range(len(data)):
    for j in range(len(tokens[i])):
        if tokens[i][j].isupper():
            tokens[i][j] = tokens[i][j].title()
re_data['text4model'] = tokens
re_data['text4model'] = re_data['text4model'].apply(lambda x: ' '.join(x))

In [32]:
re_data['text4model'] = re_data['text4model'].str.replace('\.\.\.', ' ').str.replace('\s+', ' ', regex=True)

C:\Users\kanya\AppData\Local\Temp/ipykernel_269280/3536642181.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  re_data['text4model'] = re_data['text4model'].str.replace('\.\.\.', ' ').str.replace('\s+', ' ', regex=True)


In [33]:
re_data['text4model'] = re_data['text4model'].apply(lambda x: ' '.join([token for token in x.split() if len(token) >= 2]))

In [ ]:
for text in re_data['text4model']:
    doc = nlp(text)
    result = " ".join([ent.lemma_ for ent in doc])
    lemma.append(result)

In [ ]:
re_data['text4model'] = lemma

In [ ]:
from nltk.corpus import stopwords
stop = set(stopwords.words())
re_data['text4model'] = re_data['text4model'].apply(lambda x: ' '.join([token for token in x.split() if token not in stop]))
re_data['text4model'] = re_data['text4model'].apply(lambda x: ' '.join([token for token in x.split() if len(token) >= 3]))
import string
for i in range(len(data)):
    re_data['text4model'][i] = re_data['text4model'][i].translate(str.maketrans('', '', string.punctuation))
re_data['text4model'] = re_data['text4model'].str.replace('\s+', ' ', regex=True)

In [38]:
re_data['id'] = re_data.index
re_data.to_csv('C:/Users/kanya/Desktop/ttparagraph_addmob.txt.gz', encoding = 'utf8', index = False)

In [12]:
sample_list = ['KI_737_AutoPNP_856753408.txt',
               'KI_517_Saki_863856837.txt',
               'KI_578_SmartDataWeb_1686069537.txt',
               'KI_320_IUNO_1670745120.txt',
               'KI_690_UHCI_869843966.txt',
               'Ling_126_AkaTex_873190904.txt',
               'Ling_147_BIOGRAPHE_776146335.txt',
               'Ling_207_Literaturkritik_487390423.txt',
               'Ling_161_TextGridVernetzteForschungsumgebung_768470994.txt',
               'Ling_196_DeutschIsraelischer_667762345.txt',
               'MuWi_014_GlobalMusic2one_719934826.txt',
               'MuWi_120_InnoRegio Musicon Valley_487476190.txt',
               'MuWi_051_Kompetenznetzwerk QM & LE_1015232051.txt',
               'MuWi_049_MuBiKi_1024722414.txt',
               'MuWi_034_DysTract_1693406810.txt']

In [94]:
re_data[re_data['file'].isin(sample_list)].to_excel('C:/Users/kanya/Desktop/impact_paragraph.xlsx', encoding = 'utf8', index = False)

In [14]:
pd.read_excel('C:/Users/kanya/Desktop/impact_paragraph.xlsx')

,text,Domain,Projects,file,id,Impact
0,.1 Aufgabenstellung Gegenstand und Ziel des Fo...,KI_PDFs,IUNO,KI_320_IUNO_1670745120.txt,62017,1
1,"Copyright  IUNO 2015-2018 sicherzustellen, be...",KI_PDFs,IUNO,KI_320_IUNO_1670745120.txt,62018,1
2,Copyright  IUNO 2015-2018 praktischen Entwick...,KI_PDFs,IUNO,KI_320_IUNO_1670745120.txt,62019,1
3,.0 dar. Um den Status der Informationssicherhe...,KI_PDFs,IUNO,KI_320_IUNO_1670745120.txt,62020,1
4,Copyright  IUNO 2015-2018 Das Teilvorhaben de...,KI_PDFs,IUNO,KI_320_IUNO_1670745120.txt,62021,1
...,...,...,...,...,...,...
3171,Für eine Musikakademie oder sogar Landesmusika...,MuWi_PDFs,InnoRegio Musicon Valley,MuWi_120_InnoRegio Musicon Valley_487476190.txt,180141,0
3172,II. 3. des während der Durchführung des Vorhab...,MuWi_PDFs,InnoRegio Musicon Valley,MuWi_120_InnoRegio Musicon Valley_487476190.txt,180142,0
3173,"Ergebnisse von dritter Seite, die für die Durc...",MuWi_PDFs,InnoRegio Musicon Valley,MuWi_120_InnoRegio Musicon Valley_487476190.txt,180143,0
3174,II. 4. der erfolgten oder geplanten Veröffentl...,MuWi_PDFs,InnoRegio Musicon Valley,MuWi_120_InnoRegio Musicon Valley_487476190.txt,180144,0
